In [25]:
import pandas as pd
import numpy as np
import xgboost as xgb
#Functions for xgboost
from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score
from sklearn import datasets
from sklearn import svm

#!pip install eli5

from sklearn.feature_extraction import DictVectorizer
from sklearn.pipeline import make_pipeline

In [7]:
colnames = ['encounterID','age','edVisitCountPast3months','admissionMonth','admissionBetweenThuAndSat',
            'edTimeBeforeAdmission','sex','ethnicity','admitSource','meanLOSPast24months','inpatientAdmitCountPast3months',
            'meanTemperaturePast12months','inpatientAdmitCountPast6months','inpatientAdmitCountPast12months',
            'hospitalAdmitCountPast3months','hospitalAdmitCountPast6months','edVisitCountPast6months',
            'medianTemperaturePast12months','meanHeartRatePast12months','actualLOS']

In [8]:
# Remove the variables which are not needed for the model
newCols = colnames[1:]
print(newCols)

['age', 'edVisitCountPast3months', 'admissionMonth', 'admissionBetweenThuAndSat', 'edTimeBeforeAdmission', 'sex', 'ethnicity', 'admitSource', 'meanLOSPast24months', 'inpatientAdmitCountPast3months', 'meanTemperaturePast12months', 'inpatientAdmitCountPast6months', 'inpatientAdmitCountPast12months', 'hospitalAdmitCountPast3months', 'hospitalAdmitCountPast6months', 'edVisitCountPast6months', 'medianTemperaturePast12months', 'meanHeartRatePast12months', 'actualLOS']


In [9]:
data = pd.read_csv("mimicLOSKenSciFeaturesData.csv")

In [10]:
subset = data[newCols]
#drop categorical variables and actualLOS for now
subset = subset.drop(['sex', 'ethnicity', 'admitSource', 'actualLOS'], axis=1)

In [11]:
subset.head()

,age,edVisitCountPast3months,admissionMonth,admissionBetweenThuAndSat,edTimeBeforeAdmission,meanLOSPast24months,inpatientAdmitCountPast3months,meanTemperaturePast12months,inpatientAdmitCountPast6months,inpatientAdmitCountPast12months,hospitalAdmitCountPast3months,hospitalAdmitCountPast6months,edVisitCountPast6months,medianTemperaturePast12months,meanHeartRatePast12months
0,0,0,10,0,NaN,NaN,0,NaN,0,0,0,0,0,NaN,NaN
1,48,0,6,0,NaN,NaN,0,NaN,0,0,0,0,0,NaN,NaN
2,76,0,6,0,0.22,NaN,0,NaN,0,0,0,0,0,NaN,NaN
3,53,0,5,1,0.26,NaN,0,NaN,0,0,0,0,0,NaN,NaN
4,54,0,1,0,0.22,2.47,0,NaN,0,0,0,0,0,NaN,92.5


In [12]:
#Load data into DMatrix
#Deals with categorical variables. Function converts variables into dummy numerical variables that xgboost can use
sex_encoded = pd.get_dummies(data['sex'])
ethnicity_encoded = pd.get_dummies(data['ethnicity'])
admitSource_encoded = pd.get_dummies(data['admitSource'])


#join converted variables with subset
frames = [subset, sex_encoded, ethnicity_encoded, admitSource_encoded, data['actualLOS']]
result = pd.concat(frames, axis=1)
result.head()

,age,edVisitCountPast3months,admissionMonth,admissionBetweenThuAndSat,edTimeBeforeAdmission,meanLOSPast24months,inpatientAdmitCountPast3months,meanTemperaturePast12months,inpatientAdmitCountPast6months,inpatientAdmitCountPast12months,...,WHITE - BRAZILIAN,WHITE - EASTERN EUROPEAN,WHITE - OTHER EUROPEAN,WHITE - RUSSIAN,emd,hosp-trans,mp,nursing,other,actualLOS
0,0,0,10,0,NaN,NaN,0,NaN,0,0,...,0,0,0,0,0,0,1,0,0,0.37
1,48,0,6,0,NaN,NaN,0,NaN,0,0,...,0,0,0,0,0,1,0,0,0,3.38
2,76,0,6,0,0.22,NaN,0,NaN,0,0,...,0,0,0,0,1,0,0,0,0,6.16
3,53,0,5,1,0.26,NaN,0,NaN,0,0,...,0,0,0,0,1,0,0,0,0,0.59
4,54,0,1,0,0.22,2.47,0,NaN,0,0,...,0,0,0,0,1,0,0,0,0,1.36


In [13]:
#Drops null values in actualLOS Column
result = result.dropna(subset=['actualLOS'])
len(result)



58868

In [14]:
#Take only top 10% of rows for time sake. CHANGE LATER
result = result.iloc[0:600,:]
len(result)

600

In [15]:
#Split into training and target sets (acutalLOS is target)

training = result.iloc[:, 0:62]
target = result.iloc[:,63:]
target.head()

,actualLOS
0,0.37
1,3.38
2,6.16
3,0.59
4,1.36


In [16]:
#Ask about these variabeles
seed = 7
test_size = 0.33

X_train, X_test, y_train, y_test = train_test_split(training, target, test_size=test_size, random_state=seed)

In [17]:
model = XGBClassifier()
model.fit(X_train, y_train)

print(model)

/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:95: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:128: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='multi:softprob', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)


In [18]:
#In loop take mae of each cross validation step
X_np = np.array(training)
Y_np = np.array(target)

scores = cross_val_score(model, X_np, Y_np, cv=5, scoring='mean_absolute_error')
scores

/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:95: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:128: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/anaconda3/lib/python3.6/site-packages/sklearn/metrics/scorer.py:100: DeprecationWarning: Scoring method mean_absolute_error was renamed to neg_mean_absolute_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check

array([-3.61925   , -4.3855    , -6.33066667, -5.27116667, -4.90208333])

In [19]:
#Make predictions for test data
y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]

/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [20]:
#Evaluate Predictions
MAE = mean_absolute_error(y_test, predictions)
#accuracy = accuracy_score(y_test, predictions, normalize=False)
print(MAE)

4.172727272727272


In [21]:
#Average MAE
print(sum(scores)/len(scores) *-1)

4.9017333333333335


In [42]:
#Acutal LOS Mean (for top 600 rows)
print(sum(result['actualLOS'])/len(result['actualLOS']))

5.366349999999997


In [39]:
#Show Weights
from eli5 import show_weights
vec = DictVectorizer()

show_weights(model, vec=vec)

TypeError: 'str' object is not callable

In [38]:
#Eli5 Prediction
from eli5 import show_prediction
show_prediction(model, X_test.iloc[1], vec=vec, show_feature_values=True)


TypeError: 'str' object is not callable

In [36]:
X_test.iloc[1]

age                                                         56.0
edVisitCountPast3months                                      0.0
admissionMonth                                              11.0
admissionBetweenThuAndSat                                    0.0
edTimeBeforeAdmission                                        NaN
meanLOSPast24months                                          NaN
inpatientAdmitCountPast3months                               0.0
meanTemperaturePast12months                                  NaN
inpatientAdmitCountPast6months                               0.0
inpatientAdmitCountPast12months                              0.0
hospitalAdmitCountPast3months                                0.0
hospitalAdmitCountPast6months                                0.0
edVisitCountPast6months                                      0.0
medianTemperaturePast12months                                NaN
meanHeartRatePast12months                                    NaN
female                   